In [1]:
import sys
sys.path.insert(0, '../')

import numpy as np
# set numpy precision to 3 decimal places
np.set_printoptions(precision=3, suppress=True)

This tutorial is a quick introduction to [FILLNAME](https://jupyter.org/), and introduces the following topics:
* Evaluating excitation energies
* Calculating the transition density matrix for the oscillator strength
* Computing the electron affinities


In [2]:
# Import the package
from eomee.tools import find_datafiles, hartreefock_rdms, spinize
from eomee import EOMExc, EOMExc0
from eomee.ionization import EOMIP, EOMIPc, EOMIPa

### Load data

All high-level methods in this package require:
* the electronic Hamiltonian
* the reference wavefunction 

The fist one must be specified through the one- and two-electron integrals and the second through the 1- and 2-electron reduced density matrices (RDMs).

We will use the Berillium atom as the model system. First we will evaluate the spectroscopic properties for the Hartree-Fock reference and afterwards for the Full configuration interaction model.

In this exmple, the Hamiltonian elements will be loaded from Numpy arrays, but they could be loaded from a FCIDUMP file format generated by an external quantum chemistry package using [IOData](https://iodata.readthedocs.io/en/latest/).

In [3]:
# 1) Load the electron integrals 
# nelecs = (2, 2) # number of alpha, beta electrons for Be
# one_int = np.load('../eomee/test/data/be_sto3g_oneint.npy')
# two_int = np.load('../eomee/test/data/be_sto3g_twoint.npy')
# oneint = spinize(one_int)
# twoint = spinize(two_int)

nelecs = (8, 8)
one_int = np.load(find_datafiles('c2h4_631g_oneint.npy'))
two_int = np.load(find_datafiles('c2h4_631g_twoint.npy'))
oneint = spinize(one_int)
twoint = spinize(two_int)

Our first reference will be the Hartree-Fock wavefunction. The 1- and 2-electron reduced density matrices can be computed using the helper function `hartreefock_rdms` from the `tools` module.

In [4]:
# 2) Get HF 1- and 2-RDMs
nbasis = one_int.shape[0]
rdm1, rdm2 = hartreefock_rdms(nbasis, *nelecs)

### Ionization Potentials

To compute the ionization potentials (IP) use one of the method `EOMIP`, `EOMIPc` or `EOMIPa`. The istances of these class encode the generalized eigenvalue problem that must be solved to get the transition energies to remove one electron from the ground state and the corresponding wavefunctions.

In [5]:
ekt = EOMIP(oneint, twoint, rdm1, rdm2)

print("Generalized Fock matrix:\n", ekt.lhs)

Generalized Fock matrix:
 [[11.225  0.    -0.05  ... -0.    -0.    -0.   ]
 [ 0.    11.225  0.002 ... -0.    -0.    -0.   ]
 [-0.05   0.002  0.669 ... -0.    -0.    -0.   ]
 ...
 [-0.    -0.    -0.    ... -0.    -0.    -0.   ]
 [-0.    -0.    -0.    ... -0.    -0.    -0.   ]
 [-0.    -0.    -0.    ... -0.    -0.    -0.   ]]


### Electronic Excited States

To get the excitation energies use the method `EOMExc` or `EOMExc0`. The istance of this class encodes the generalized eigenvalue problem that must be solved to get the excitation energies and the corresponding wavefunctions.

In [6]:
myeom = EOMExc(oneint, twoint, rdm1, rdm2)

print("Diagonal elements of orbital Hessian matrix:\n", np.diag(myeom.lhs))

Diagonal elements of orbital Hessian matrix:
 [0. 0. 0. ... 0. 0. 0.]


All the excitation energies can be computed at once with the `solve_dense` method or only a few of them with `solve_sparse`. The number of lowest excitation energies to be computed is specified by the parameter `nsols`.

In [7]:
ev, cv = myeom.solve_dense()
(e1, e2) = myeom.solve_sparse(nsols=2)[0]

print("Number of EAs: ", len(ev))
print("Two lowest EAs: ", e1, e2)

Number of EAs:  573
Two lowest EAs:  -12.413615652648494 -12.413311962207613


In [8]:
print("{0:<2s} {1:<6s}".format('n', 'EDiff.(a.u.)'))
for i, e in enumerate(ev[:5]):
    print("{0:<2} {1:<6.3}".format(i, e))

n  EDiff.(a.u.)
0  0.299 
1  0.325 
2  0.325 
3  0.325 
4  0.346 


For closed-shell systems, such as neutral Be, one can use the spin-adapted implementations `EOMEE1` and `EOMEE3` to compute the singlet and triplet excitation energies, respectively. These classes are subclasses of `EOMExc` and have the same methods.

In [9]:
from eomee.spinadapted.particlehole import EOMEE1, EOMEE3, EOMEE03

myeom = EOMEE3(oneint, twoint, rdm1, rdm2)
ev1, cv1 = myeom.solve_dense()

# Print the first 5 excitation energies
print("{0:<2s} {1:<6s}".format('n', 'EDiff.(a.u.)'))
for i, e in enumerate(ev1[:5]):
    print("{0:<2} {1:<6.3}".format(i, e))

n  EDiff.(a.u.)
0  0.325 
1  0.346 
2  0.353 
3  0.366 
4  0.39  


It is also posible to compute the one-electron transition density matrix (TDMs) for a given excited state with the `compute_tdm1` method. The TDMs are the matrix elements of the dipole operator between the ground and excited states.

In [10]:
tdm = myeom.compute_tdm1(cv1[0])

# print(cv1[0].reshape(myeom.k, myeom.k))
# print(tdm)

In [11]:
U, s, Vh = np.linalg.svd(tdm)
V = Vh.conj().T

print("s", s)
print("s^2", s**2)
print("sum s^2", sum(s**2))
# print(tdm.shape)
# print(U)
# print(V)

s [0.996 0.089 0.046 0.036 0.019 0.005 0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.   ]
s^2 [0.992 0.008 0.002 0.001 0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.   ]
sum s^2 1.0033591229546315
